cache_dir = os.environ["TRANSFORMERS_CACHE"] # creating cache folder for transformers
os.makedirs(cache_dir, exist_ok=True)
print(f"Transformers cache directory set to: {cache_dir}")

import os
os.environ["TRANSFORMERS_CACHE"] = "/Users/jei/cache/huggingface/hub"

import transformers
from transformers import XLNetTokenizer

import os
print(f"Transformers cache location: {os.getenv("TRANSFORMERS_CACHE", "Default cache location")}")

import torch
print(f"PyTorch version: {torch.__version__}")

import os

# Set custom cache directory
os.environ["TRANSFORMERS_CACHE"] = "/Users/jei/cache/huggingface/hub"
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, pipeline

# Load and preprocess data
file_path = "lyrics_combined.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")
lyrics = df["A"].dropna()

def clean_text(text):
    import re
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

lyrics = lyrics.apply(clean_text)

with open("lyrics_dataset.txt", "w") as f:
    f.write('\n'.join(lyrics))

# Load TinyBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=1)

# Tokenize data
def tokenize_text(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    tokenized = [tokenizer(line, truncation=True, padding="max_length", max_length=128)["input_ids"] for line in lines]
    return tokenized

tokenized_data = tokenize_text("lyrics_dataset.txt")

class LyricsDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx])

dataset = LyricsDataset(tokenized_data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        batch = batch.to(device)
        labels = torch.zeros(batch.size(0), dtype=torch.long).to(device)  # Example labels
        outputs = model(batch, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Save the model and tokenizer
model.save_pretrained("./fine_tuned_tinybert")
tokenizer.save_pretrained("./fine_tuned_tinybert")

# Example inference using TinyBERT
text_classifier = pipeline("text-classification", model="./fine_tuned_tinybert", tokenizer=tokenizer)
print(text_classifier("deez nutz"))

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/jei/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
